## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-31-13-07-13 +0000,nyt,"Deadly Israeli Strikes in Gaza, as Rafah Borde...",https://www.nytimes.com/2026/01/31/world/middl...
1,2026-01-31-13-00-05 +0000,nyt,The Latest Way to Create an Isolated Retreat: ...,https://www.nytimes.com/2026/01/31/business/de...
2,2026-01-31-13-00-00 +0000,wsj,Venezuela’s New Leader Is Trying to Run Out th...,https://www.wsj.com/world/americas/venezuelas-...
3,2026-01-31-13-00-00 +0000,latimes,"LAUSD teachers union members authorize strike,...",https://www.latimes.com/california/story/2026-...
4,2026-01-31-12-55-03 +0000,bbc,Labour pick Angeliki Stogia for Gorton by-elec...,https://www.bbc.com/news/articles/clyn5zj8750o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2468/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,58
92,ice,29
47,epstein,28
208,fed,25
41,warsh,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
30,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...,187
269,2026-01-30-17-24-50 +0000,nypost,Gold and silver plunge after news Trump will n...,https://nypost.com/2026/01/30/business/gold-si...,153
244,2026-01-30-19-01-53 +0000,nyt,Trump Picks Kevin Warsh as Next Fed Chair,https://www.nytimes.com/2026/01/30/us/politics...,138
198,2026-01-30-21-08-13 +0000,nyt,"Kevin Warsh, Trump’s Choice for Fed Chair Coul...",https://www.nytimes.com/2026/01/30/business/tr...,138
316,2026-01-30-14-39-53 +0000,nyt,"Who Is Kevin Warsh, Trump’s Pick to Lead the Fed?",https://www.nytimes.com/2026/01/30/us/politics...,137


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
30,187,2026-01-31-10-30-00 +0000,wsj,Trump’s pick for Fed chair has long attacked t...,https://www.wsj.com/economy/central-banking/ke...
56,85,2026-01-31-06-19-29 +0000,nypost,LA Olympics boss Casey Wasserman’s racy emails...,https://nypost.com/2026/01/31/us-news/la-olymp...
118,79,2026-01-31-01-25-43 +0000,bbc,Former CNN host Don Lemon charged in anti-ICE ...,https://www.bbc.com/news/articles/c24gm3dz36po...
60,69,2026-01-31-05-07-00 +0000,wsj,The Senate passed a bipartisan deal Friday to ...,https://www.wsj.com/politics/policy/senate-set...
146,45,2026-01-30-23-30-03 +0000,bbc,Luigi Mangione will not face death penalty if ...,https://www.bbc.com/news/articles/c78egdg3r0po...
149,39,2026-01-30-23-12-35 +0000,nypost,Disney CEO Bob Iger plans to step down before ...,https://nypost.com/2026/01/30/business/disney-...
65,38,2026-01-31-04-45-59 +0000,nypost,Cardozo’s Ron Naclerio wins 973rd game to set ...,https://nypost.com/2026/01/30/sports/cardozo-h...
70,34,2026-01-31-04-03-13 +0000,eltecolote,"Students, teachers walk out as San Francisco j...",https://eltecolote.org/content/en/ice-walkout-...
130,34,2026-01-31-00-19-00 +0000,bbc,One wrong move and it could all go wrong - the...,https://www.bbc.com/news/articles/c70lgwprw7ko...
82,31,2026-01-31-03-16-39 +0000,nypost,Macaulay Culkin seen for first time since deat...,https://nypost.com/2026/01/30/us-news/macaulay...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
